## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mataura,NZ,-46.1927,168.8643,40.48,90,99,3.69,overcast clouds
1,Butaritari,KI,3.0707,172.7902,83.59,71,0,16.89,clear sky
2,Cherskiy,RU,68.7500,161.3000,38.55,86,63,6.67,broken clouds
3,Ribeira Grande,PT,38.5167,-28.7000,75.60,78,40,19.57,scattered clouds
4,Palmer,US,42.1584,-72.3287,72.01,85,20,0.00,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Butaritari,KI,3.0707,172.7902,83.59,71,0,16.89,clear sky
3,Ribeira Grande,PT,38.5167,-28.7000,75.60,78,40,19.57,scattered clouds
6,Souillac,MU,-20.5167,57.5167,75.58,60,40,8.05,scattered clouds
7,Kapaa,US,22.0752,-159.3190,79.12,83,90,4.00,light rain
13,Berezovyy,RU,51.6667,135.7000,76.55,68,18,3.60,few clouds
15,Kailua,US,21.4022,-157.7394,78.78,76,75,11.50,broken clouds
17,Bambous Virieux,MU,-20.3428,57.7575,75.45,60,40,8.05,scattered clouds
19,Roswell,US,34.0232,-84.3616,76.39,93,1,0.00,mist
24,Atuona,PF,-9.8000,-139.0333,77.68,70,97,15.14,overcast clouds
39,Itarema,BR,-2.9248,-39.9167,85.48,48,65,15.03,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)
# clean_df.isnull().sum()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Butaritari,KI,3.0707,172.7902,83.59,71,0,16.89,clear sky
3,Ribeira Grande,PT,38.5167,-28.7000,75.60,78,40,19.57,scattered clouds
6,Souillac,MU,-20.5167,57.5167,75.58,60,40,8.05,scattered clouds
7,Kapaa,US,22.0752,-159.3190,79.12,83,90,4.00,light rain
13,Berezovyy,RU,51.6667,135.7000,76.55,68,18,3.60,few clouds
15,Kailua,US,21.4022,-157.7394,78.78,76,75,11.50,broken clouds
17,Bambous Virieux,MU,-20.3428,57.7575,75.45,60,40,8.05,scattered clouds
19,Roswell,US,34.0232,-84.3616,76.39,93,1,0.00,mist
24,Atuona,PF,-9.8000,-139.0333,77.68,70,97,15.14,overcast clouds
39,Itarema,BR,-2.9248,-39.9167,85.48,48,65,15.03,broken clouds


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,83.59,clear sky,3.0707,172.7902,
3,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,
6,Souillac,MU,75.58,scattered clouds,-20.5167,57.5167,
7,Kapaa,US,79.12,light rain,22.0752,-159.3190,
13,Berezovyy,RU,76.55,few clouds,51.6667,135.7000,
15,Kailua,US,78.78,broken clouds,21.4022,-157.7394,
17,Bambous Virieux,MU,75.45,scattered clouds,-20.3428,57.7575,
19,Roswell,US,76.39,mist,34.0232,-84.3616,
24,Atuona,PF,77.68,overcast clouds,-9.8000,-139.0333,
39,Itarema,BR,85.48,broken clouds,-2.9248,-39.9167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
       

    # Add the latitude and longitude to location key for the params dictionary.
    

    # Use the search term: "lodging" and our latitude and longitude.
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not Found...skipping.")
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
